# Project 2 - Machine Learning

Students: Joel Jonassi 19698
        Rui Alves 15505
Teacher: Joaquim Silva

DataSet Link : https://www.kaggle.com/datasets/camnugent/california-housing-prices

# Introduction

For this job we will analyse the California house dataset...

Will start with a contextualization about the metrics that we will use to evaluate the models.

Note: Underfiting if the training error and error testing high
Overfitting if error training is low or accurancy high and error testing low or accurancy high



## Contextualization

At this section will make a contextualization about the metrics used in this project to measure the models performance.

### RandomForest and DecisionTreeClassifier Metrics

#### Classification accuracy

The classification accuracy is the ratio of number of correct predictions to the total number of input samples.
This metric works well only if there are equal number of samples belonging to each class.

According to the "https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234", says that if we consider that there are 98% samples of class A and 2% samples of class B in our training set. Then our model can easily get 98% training accuracy by simply predicting every training sample belonging to class A.
When the same model is tested on a test set with 60% samples of class A and 40% samples of class B, then the test accuracy would drop down to 60%. Classification Accuracy is great, but gives us the false sense of achieving high accuracy.

![Result](img/ClassificationAccuracy.png)


#### Confusion Matrix

Confusion Matrix as the name suggests gives us a matrix as output and describes the complete performance of the model.
There are 4 important terms :

* True Positives : The cases in which we predicted YES and the actual output was also YES.
* True Negatives : The cases in which we predicted NO and the actual output was NO.
* False Positives : The cases in which we predicted YES and the actual output was NO.
* False Negatives : The cases in which we predicted NO and the actual output was YES.
Accuracy for the matrix can be calculated by taking average of the values lying across the “main diagonal”.

![Result](img/ConfusionMatrix.png)


#### F1 Score

F1 Score is used to measure a test’s accuracy

F1 Score is the Harmonic Mean between precision and recall. The range for F1 Score is [0, 1]. It tells you how precise your classifier is (how many instances it classifies correctly), as well as how robust it is (it does not miss a significant number of instances).

High precision but lower recall, gives you an extremely accurate, but it then misses a large number of instances that are difficult to classify. The greater the F1 Score, the better is the performance of our model. Mathematically, it can be expressed as :


F1 Score tries to find the balance between precision and recall.

![Result](img/F1Score.png)


Precision : It is the number of correct positive results divided by the number of positive results predicted by the classifier.

![Result](img/F1ScorePrecision.png)

Recall : It is the number of correct positive results divided by the number of all relevant samples (all samples that should have been identified as positive).

![Result](img/F1ScoreRecall.png)


#### Mean Absolute Error
Mean Absolute Error is the average of the difference between the Original Values and the Predicted Values. It gives us the measure of how far the predictions were from the actual output. However, they don’t gives us any idea of the direction of the error i.e. whether we are under predicting the data or over predicting the data. Mathematically, it is represented as :

![Result](img/MeanAbsoluteError.pngg)

The closer MAE is to 0, the more accurate the model is.

#### Mean Squared Error
Mean Squared Error(MSE) is quite similar to Mean Absolute Error, the only difference being that MSE takes the average of the square of the difference between the original values and the predicted values. The advantage of MSE being that it is easier to compute the gradient, whereas Mean Absolute Error requires complicated linear programming tools to compute the gradient. As, we take square of the error, the effect of larger errors become more pronounced then smaller error, hence the model can now focus more on the larger errors.

![Result](img/MeanSquaredError.pngg)



### K-Means Metrics
For cluster evaluation we use the following metrics:
* 1
* 2







### Associations Rules Metrics

In the same way as we do above, we will make a contextualization of the metrics used in this project to evaluate the the model created with Apriori Algorithms.

* Confidence ...

* Suport ...

* Lift ...

### Imports side

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
#from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder
from apyori import apriori
from sklearn import tree
from kneed import KneeLocator
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import matplotlib.cm as cm


### DataSet reading

In [3]:
df = pd.read_csv("dataset/housingDataPrepare.csv")

## Automatic Classification

Objetives:
 * Try do predict how far the house is from the beach.

### Data preparation

We consider 10 dataset attributes important to train the model to predict how far the house is near form the beach.
From the 10 attributes, we used 9 for testing and 1 for the model to predict the target:
* latitude and longitude - we use this two variables to helps ous finding the location of the houses.
* house_median_age - this attribute helps our model to predict the age of the house.
* total_rooms and total_bedrooms- we find these variables important to train the model because can tells ous how many rooms are near to the beach, as we know there are more houses/rooms for tourism near to the beach.
* population - can have a correlation between population and proximity to the beach.
* house_hold - we assume that the household can have influence of proximity to the beach or not.
* median_income and median_house_value - these two variables we assume that are essential to predict if the house is near the beach, because people with with higher purchasing power are more likely to live there due to the cost of living in these areas.
* ocean_proximity - This is the target attribute, we will use two algorithms to predict the ocean proximity namely Decision Random Forest Algorithm and Tree Classifier.



### Random Forest Algorithm

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean') 
data = df.iloc[: , :-3] # Ignore last 3 columns

X = data.iloc[ :, : -1].values # Use all columns except de last one in "data"

imp.fit(X)
X = imp.transform(X)
Y = data.iloc[ :, 9].values # Target 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=0)

#Using random forest classifier
classifier = RandomForestClassifier() 
classifier = classifier.fit(X_train, Y_train) # model creation
predicted = classifier.predict(X_test) # model evaluation

cols = ['ISLAND', 'NEAR OCEAN', 'NEAR BAY', '<1H OCEAN', 'INLAND']

#Results
print ('Confusion Matrix :')
confm = confusion_matrix(Y_test, predicted, labels=cols)
print(confm)
print('Accuracy Score :', accuracy_score(Y_test, predicted))
print('Report : ')
print(classification_report(Y_test, predicted, labels=cols, zero_division=0))

df_cm = DataFrame(confm, index=cols, columns=cols)
ax = sns.heatmap(df_cm, cmap='Oranges', annot=True, fmt="d")


conv_to_num = {
    'ISLAND': 1,
    'NEAR OCEAN': 2,
    'NEAR BAY': 3,
    '<1H OCEAN': 4,
    'INLAND': 5
}

def convert_num(name):
    return conv_to_num[name]

mae_y_true = list(map(convert_num, Y_test))
mae_y_pred = list(map(convert_num, predicted))
print(f"Mean Absolute Error: {mean_absolute_error(mae_y_true, mae_y_pred)}")

### Decision Tree Classifier

In [ ]:
DTC = DecisionTreeClassifier().fit(X_train, Y_train) # model creation
predicted = DTC.predict(X_test) # model evaluation
print('confusion Matrix')
confm = confusion_matrix(Y_test, predicted, labels=cols)
print(confm)
print('Accuracy Score : ', accuracy_score(Y_test, predicted))
print("Report: ")
print(classification_report(Y_test, predicted))

mae_y_true = list(map(convert_num, Y_test))
mae_y_pred = list(map(convert_num, predicted))
print(f"Mean Absolute Error: {mean_absolute_error(mae_y_true, mae_y_pred)}")

df_cm = DataFrame(confm, index=cols, columns=cols)
ax = sns.heatmap(df_cm, cmap='Oranges', annot=True, fmt="d")

In [ ]:
feature_names = ["longitude"	,"latitude","housing_median_age"	,"total_rooms"	,"total_bedrooms"	,"population",	"households",	"median_income"	,"median_house_value"]
target_names = "ocean_proximity"
plt.figure(figsize=(60,60))
tree.plot_tree(DTC, filled=True, feature_names=feature_names, class_names=target_names,fontsize=6)


## Results Analysis
At this section we will discuss a classification Accuracy of RandomForest and DecisionTreeClassifier Algorithms. 

### Analysis
As we can see in confusion matrix of these two algorithms we observe that the ISLAND in the RandomForest algorithm isn't classified correctly although in DecisionTreeClassifier algorithm it was, even so this algorithm has a worst accuracy score than the RandomForest algorithm.

Observing the Mean Absolute Error we can conclude how far we are from classifying everything correctly.

RandomForest Confusion Matrix

![Result](img/RandomForestConfM.png)


Accuracy Score : 0.9751291989664083

Mean Absolute Error: 0.03504521963824289


DecisionTree Confusion Matrix

![Result](img/DecisionTreeConfM.png)

Accuracy Score :  0.9748062015503876

Mean Absolute Error: 0.036337209302325583

## Cluster with K-Means

Objetives:
 * Classify the zones where the people with the highest purchasing power reside. 

### Data prepare

For the algorithm K-Means we just use 3 attributes of the dataset namely:

* median_house_value - allow to make clusters with house values.

* latitude and longitude - To find where the purchasing power people live.


In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

X = df.loc[:, ["median_house_value", "latitude", "longitude"]]
X.head()



In [ ]:
#Elbow
cost =[]
for i in range(1, 11):
    KM = KMeans(n_clusters = i, max_iter = 100)
    KM.fit(X)
     
    # calculates squared error
    # for the clustered points
    cost.append(KM.inertia_)    
 
# plot the cost against K values
plt.plot(range(1, 11), cost, color ='g', linewidth ='3')
plt.xlabel("Value of K")
plt.ylabel("Squared Error (Cost)")
plt.show() # clear the plot

kl = KneeLocator(range(1, 11), cost, curve="convex", direction="decreasing" )
print(f"The optimal K is {kl.elbow}")

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=3)

X["Cluster"] = kmeans.fit_predict(X)

X["Cluster"] = X["Cluster"].astype("category")

X.head()


A scatter plot that shows the geographic distribution of the clusters. It seems like the algorithm has created separate segments for higher-income areas.

In [ ]:
%matplotlib widget

sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x  = X["latitude"]
y  = X["longitude"]
z  = X["median_house_value"]
ax.set_xlabel("latitude")
ax.set_ylabel("longitude")
ax.set_zlabel("median_house_value")
ax.scatter(x, y, z)
plt.show()

#2d
#sns.relplot(x="longitude", y="latitude", hue="Cluster", data=X, height=6,);

sns.relplot(
    x="longitude", y="latitude", hue="Cluster", data=X, height=6,
)

The target in this dataset is median_house_value (median house value). These box-plots show the distribution of the target within each cluster.

In [ ]:
X["median_income"] = df["median_income"]
sns.catplot(x="median_income", y="Cluster", data=X, kind="boxen", height=9)

X["median_income"] = df["median_income"]
sns.catplot(x="median_income", y="Cluster", data=X, kind="boxen", height=9)
## Results Analysis
At this section we will discuss the performance of the K-Means algorithm.

## Association rules

Objetives:
* Knowing the income, the age of house and price of house, associate age of house and price of house and income of someone with a house price.

### Data Preparation
For this algorithm "Apriori" we prepare the data for association rules. We Use 3 attributes we consider important to complete our objective.
First of all we create a interval of these attributes, labelling each interval by its representing information.
For the attribute median_income, we create 5 labels intervals of 2. The labels are: Very Low Income, Low Income, Medium Income, High Income and Very High Income.
In excel we use the function below to do this transformation.
```bash
=SE(H2<=2,"Very Low Income",SE(E(2<H2,H2<=4),"Low Income",SE(E(4<H2,H2<=6),"Medium Income",SE(E(6<H2,H2<=8),"High Income","Very High Income"))))
```

For the attribute housing_median_age we label also with 5  labels namely: Very Recent House, Recent House, Medium Aged House, Older House and Very Older House.

```bash
=SE(I2<=11,"Very Recent House",SE(E(11<I2,I2<=22),"Recent House",SE(E(22<I2,I2<=33),"Medium Aged House",SE(E(33<I2,I2<=44),"Older House","Very Older House"))))
```

In the same way we label the attribute median_house_value with the following labels: Very Cheap House, Cheap House, Medium Priced House, Expensive House and Very Expensive House.
```bash
=SE(I2<=90000,"Very Cheap House",SE(E(90000<I2,I2<=180000),"Cheap House",SE(E(180000<I2,I2<=270000),"Medium Priced House",SE(E(270000<I2,I2<=360000),"Expensive House","Very Expensive House"))))
```

In [4]:
# Data Segragation
data = df.iloc[: , :] # All data
data = df.iloc[:, 9 :]
print(data)

      ocean_proximity median_income_class median_house_value_class  \
0            NEAR BAY     Very High Icome     Very Expensive House   
1            NEAR BAY     Very High Icome          Expensive House   
2            NEAR BAY          High Icome          Expensive House   
3            NEAR BAY           Low Icome          Expensive House   
4            NEAR BAY           Low Icome          Expensive House   
...               ...                 ...                      ...   
20635          INLAND     Very Low Income         Very Cheap House   
20636          INLAND           Low Icome         Very Cheap House   
20637          INLAND     Very Low Income              Cheap House   
20638          INLAND     Very Low Income         Very Cheap House   
20639          INLAND           Low Icome         Very Cheap House   

      hosing_median_age_class  
0                 Older House  
1                Recent House  
2            Very Older House  
3            Very Older House  

In [5]:
records = []
for i in range(0, len(data)):
    records.append([str(data.values[i, j]) for j in range(0, 4)])

In [6]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, max_length=3) # creation of the model
association_results = list(association_rules)

In [27]:
def inspect(association_results):
    lhs         = [tuple(result[2][0][0])[0] for result in association_results]
    rhs         = [tuple(result[2][0][1])[0] for result in association_results]
    support    = [result[1] for result in association_results]
    confidence = [result[2][0][2] for result in association_results]
    lift       = [result[2][0][3] for result in association_results]
    return list(zip(lhs, rhs, support, confidence, lift))

output_DataFrame = pd.DataFrame(inspect(association_results), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
print("Total number of rules mined = " , len(association_results))
output_DataFrame.nlargest(n = 90, columns = 'Support')

Total number of rules mined =  36


,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
3,Very Cheap House,Very Low Income,0.055717,0.427350,3.589952
20,Very Cheap House,INLAND,0.046705,0.358231,5.948424
2,NEAR BAY,Very Older House,0.043556,0.392576,3.234642
18,Very Cheap House,INLAND,0.035126,0.269417,3.240535
22,NEAR BAY,Very Older House,0.034399,0.310044,3.422086
0,Expensive House,High Icome,0.031686,0.256974,3.172221
1,High Icome,Very Expensive House,0.031347,0.386962,3.374267
4,Very Expensive House,Very High Icome,0.030136,0.262780,7.860546
19,Very Cheap House,INLAND,0.029845,0.228911,3.977043
5,High Icome,<1H OCEAN,0.021560,0.266148,3.749694


### Result Analysis

At this section we will evaluate the Apriori model.

As we can the Apriori algorithm, satisfy our objectives to make a association between these three attributes.

image result

Now we will make a deep evaluation of the model:
In evaluating process of the Apriori model we consider the support, confidence and Lift values.

Rule: High Income -> Expensive House
Suport : 
Confidence : 
lift : 

Due to de dataset length we observe that the creation of candidates at the first time we run the algorithm it took some time, we conclude that if we have many atributes to make a association it require much processing power.

# Conclusion

Finally this job ...